In [ ]:
import os
from pydub import AudioSegment

def mix_audio(input_dir, output_dir, noise_file, mix_levels, skip_existing=True):
    print(f"Noise file: {noise_file}")
    # load noise
    audio_noise = AudioSegment.from_file(noise_file)
    noise_filename_base, _ = os.path.splitext(os.path.basename(noise_file))

    for root, dirs, files in os.walk(input_dir):
        # sort directories alphabetically
        dirs.sort()
        directory = os.path.basename(root)
        print(f"Processing directory: {directory}")
        for file in files:
            if file.endswith('.wav'):
                # load audio
                audio_in = AudioSegment.from_file(os.path.join(root, file))
                # mix and save file for every level
                for level in mix_levels:
                    output_file = os.path.join(output_dir, f"{noise_filename_base}_{int(level*100)}", directory, file)
                    if not (skip_existing and os.path.exists(output_file)):
                        mixed_audio = audio_in.overlay(audio_noise.apply_gain((1 - level) * -30), loop=True) # reduce by up to 30 dB
                        os.makedirs(os.path.dirname(output_file), exist_ok=True)
                        mixed_audio.export(output_file, format="wav")
                    

input_dir = os.path.join("data", "audio","Speech Commands")
output_dir = os.path.join("data", "audio", "Speech Commands_noise")

noise_file = os.path.join('data', 'audio', '_background_noise_', 'white_noise.wav')
mix_levels = [.25]  # [.25, .5], etc

mix_audio(input_dir, output_dir, noise_file, mix_levels, skip_existing=True)